In [21]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data= pd.read_csv('/content/heart_disease_uci.csv')

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [23]:
data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [24]:
data.describe()

,id,age,trestbps,chol,thalch,oldpeak,ca,num
count,920.000000,920.000000,861.000000,890.000000,865.000000,858.000000,309.000000,920.000000
mean,460.500000,53.510870,132.132404,199.130337,137.545665,0.878788,0.676375,0.995652
std,265.725422,9.424685,19.066070,110.780810,25.926276,1.091226,0.935653,1.142693
min,1.000000,28.000000,0.000000,0.000000,60.000000,-2.600000,0.000000,0.000000
25%,230.750000,47.000000,120.000000,175.000000,120.000000,0.000000,0.000000,0.000000
50%,460.500000,54.000000,130.000000,223.000000,140.000000,0.500000,0.000000,1.000000
75%,690.250000,60.000000,140.000000,268.000000,157.000000,1.500000,1.000000,2.000000
max,920.000000,77.000000,200.000000,603.000000,202.000000,6.200000,3.000000,4.000000


In [25]:
data.isnull().sum()


,0
id,0
age,0
sex,0
dataset,0
cp,0
trestbps,59
chol,30
fbs,90
restecg,2
thalch,55


# DATA PRE PROCESSING


* **FILLING NUMERICAL VALUES**


**FILLING TRESTBPS**

In [26]:
# filling using age and sex only

In [27]:
bins=[20,30,40,50,60,70,80]
labels =['20s', '30s','40s','50s','60s','70s']
data['age_bin'] = pd.cut(data['age'], bins=bins, labels=labels, right =False)


In [28]:
# criteria=['age_bin','sex','dataset', 'cp','num']
# data['chol'] = data['chol'].fillna(data.groupby(criteria)['chol'].transform('median'))
# data['trestbps']= data['trestbps'].fillna(data.groupby(criteria)['trestbps'].transform('median'))
# print(data['chol'].isnull().sum())
# print(data['trestbps'].isnull().sum())





In [29]:
# less_crietria=  ['age_bin','sex']
# data['chol'] = data['chol'].fillna(data.groupby(less_crietria)['chol'].transform('median'))
# data['trestbps'] = data['trestbps'].fillna(data.groupby(less_crietria)['trestbps'].transform('median'))
# print(data['chol'].isnull().sum())
# print(data['trestbps'].isnull().sum())


In [30]:
# print(data['thalch'].isnull().sum())
# criteria=['age_bin','sex','dataset', 'cp','num','trestbps','chol']

# data['thalch'] = data['thalch'].fillna(data.groupby(criteria)['thalch'].transform('median'))
# print(data['thalch'].isnull().sum())




***CASACADING SEQUENTIAL CUT***

In [31]:
import numpy as np
from itertools import combinations

In [32]:
import numpy as np
from itertools import combinations

def best_match_fill(df, target_col, criteria_list):
    """
    Finds missing values in target_col by finding the most specific
    matching group possible using the criteria_list.
    """
    mi = df[df[target_col].isnull()].index
    if len(mi) == 0:
        return df

    for idx in mi:
        found = False
        for r in range(len(criteria_list), 0, -1):
            for combo in combinations(criteria_list, r):
                combo = list(combo)
                row_vals = df.loc[idx, combo]

                match_mask = (df[combo] == row_vals).all(axis=1) & df[target_col].notnull()
                peers = df[match_mask]

                if not peers.empty:
                    df.at[idx, target_col] = peers[target_col].median()
                    found = True
                    break
            if found: break

        if not found:
            df.at[idx, target_col] = df[target_col].median()

    return df

In [33]:
criteria = ['age_bin', 'sex', 'dataset', 'cp', 'num']

numerical_cols = ['trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
for col in numerical_cols:
    data = best_match_fill(data, col, criteria)

    bin_name = f"{col}_bin"

    if col =='ca':
      data[bin_name] = data[col]
    elif col == 'oldpeak':
        data[bin_name] = pd.cut(data[col], bins=[-np.inf, 0, 1, 2, 3, np.inf], labels=['0', '0-1', '1-2', '2-3', '3+'])
    else:
        data[bin_name] = (data[col] // 10) * 10

    criteria.append(bin_name)

print(data[numerical_cols].isnull().sum())


trestbps    0
chol        0
thalch      0
oldpeak     0
ca          0
dtype: int64


***CATEGORICAL DATA***

In [34]:
def best_match_fill_categorical(df, target_col, criteria_list):
    """
    Finds missing values in target_col by finding the most specific
    matching group possible using the criteria_list, then using the mode.
    """
    mi = df[df[target_col].isnull()].index
    if len(mi) == 0:
        return df

    for idx in mi:
        found = False
        # Try decreasing levels of specificity (from all criteria down to 1)
        for r in range(len(criteria_list), 0, -1):
            for combo in combinations(criteria_list, r):
                combo = list(combo)
                row_vals = df.loc[idx, combo]

                # Match peers with the same criteria values that are NOT null in target_col
                match_mask = (df[combo] == row_vals).all(axis=1) & df[target_col].notnull()
                peers = df[match_mask]

                if not peers.empty:
                    mode_series = peers[target_col].mode()
                    if not mode_series.empty:
                        df.at[idx, target_col] = mode_series[0]
                        found = True
                        break
            if found: break

        # Fallback to global mode if no matches found
        if not found:
            global_mode = df[target_col].mode()
            if not global_mode.empty:
                df.at[idx, target_col] = global_mode[0]

    return df

In [35]:
cat_cols =['fbs','restecg','exang','slope','thal']
for col in cat_cols:
  data= best_match_fill_categorical(data,col,criteria)

data.isnull().sum()

,0
id,0
age,0
sex,0
dataset,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalch,0


encoding


In [36]:
#binary
df =data.copy()

df['sex']=df['sex'].map({'Male':1,'Female':0})
df['fbs']=df['fbs'].astype(int)
df['exang']=df['exang'].astype(int)


In [46]:
for col in df.columns:
    if df[col].dtype.name == 'category':
        df[col] = df[col].cat.codes

In [47]:
# nominal_cols = ['dataset', 'cp', 'restecg', 'slope', 'thal']
# df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)

# # 4. Final Cleanup: Ensure everything is a number
# df = df.astype(int)

df.head()

,id,age,sex,trestbps,chol,fbs,thalch,exang,oldpeak,ca,...,dataset_VA Long Beach,cp_atypical angina,cp_non-anginal,cp_typical angina,restecg_normal,restecg_st-t abnormality,slope_flat,slope_upsloping,thal_normal,thal_reversable defect
0,1,63,1,145.0,233.0,1,150.0,0,2.3,0.0,...,False,False,False,True,False,False,False,False,False,False
1,2,67,1,160.0,286.0,0,108.0,1,1.5,3.0,...,False,False,False,False,False,False,True,False,True,False
2,3,67,1,120.0,229.0,0,129.0,1,2.6,2.0,...,False,False,False,False,False,False,True,False,False,True
3,4,37,1,130.0,250.0,0,187.0,0,3.5,0.0,...,False,False,True,False,True,False,False,False,True,False
4,5,41,0,130.0,204.0,0,172.0,0,1.4,0.0,...,False,True,False,False,False,False,False,True,True,False


In [44]:
# 1. Start with a fresh copy of your cleaned 'data' variable
# This prevents the KeyError if you run the cell multiple times
df_final = data.copy()

# 2. Handle Binary Columns (Sex, FBS, Exang)
# We use .get() or check if they exist to avoid errors
if 'sex' in df_final.columns:
    df_final['sex'] = df_final['sex'].map({'Male': 1, 'Female': 0})
df_final['fbs'] = df_final['fbs'].astype(int)
df_final['exang'] = df_final['exang'].astype(int)

# 3. Automatically encode category-type columns (age_bin, oldpeak_bin)
for col in df_final.columns:
    if str(df_final[col].dtype) == 'category':
        df_final[col] = df_final[col].cat.codes

# 4. One-Hot Encoding for Nominal columns
# We only include columns that actually exist in the dataframe right now
nominal_cols = ['dataset', 'cp', 'restecg', 'slope', 'thal']
existing_nominal = [c for c in nominal_cols if c in df_final.columns]

if existing_nominal:
    df_final = pd.get_dummies(df_final, columns=existing_nominal, drop_first=True)

# 5. Final Cleanup: Ensure everything is a number for the model
# We drop 'id' because it's just a row counter and not a medical feature
if 'id' in df_final.columns:
    df_final = df_final.drop(columns=['id'])

df_final = df_final.astype(float)

print("Encoding successful! Current columns:", df_final.columns.tolist())
df_final.head()

Encoding successful! Current columns: ['age', 'sex', 'trestbps', 'chol', 'fbs', 'thalch', 'exang', 'oldpeak', 'ca', 'num', 'age_bin', 'trestbps_bin', 'chol_bin', 'thalch_bin', 'oldpeak_bin', 'ca_bin', 'dataset_Hungary', 'dataset_Switzerland', 'dataset_VA Long Beach', 'cp_atypical angina', 'cp_non-anginal', 'cp_typical angina', 'restecg_normal', 'restecg_st-t abnormality', 'slope_flat', 'slope_upsloping', 'thal_normal', 'thal_reversable defect']


,age,sex,trestbps,chol,fbs,thalch,exang,oldpeak,ca,num,...,dataset_VA Long Beach,cp_atypical angina,cp_non-anginal,cp_typical angina,restecg_normal,restecg_st-t abnormality,slope_flat,slope_upsloping,thal_normal,thal_reversable defect
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


# **Feature scaling**

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


helper_bins = ['age_bin', 'trestbps_bin', 'chol_bin', 'thalch_bin', 'oldpeak_bin', 'ca_bin']

existing_bins = [bin_col for bin_col in helper_bins if bin_col in df_final.columns]

X = df_final.drop(columns=['num'] + existing_bins)
y = df_final['num']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data is ready for training!")
print(f"Training features shape: {X_train_scaled.shape}")
print(f"Testing features shape: {X_test_scaled.shape}")

Data is ready for training!
Training features shape: (736, 21)
Testing features shape: (184, 21)


In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

param_grid_rf = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [6, 9, 11, None],
    'min_samples_split': [4, 7, 10],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=2026)


grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=1)

grid_search_rf.fit(X_train_scaled, y_train)

print(f"Best RF Parameters: {grid_search_rf.best_params_}")
print(f"Best CV Accuracy: {grid_search_rf.best_score_:.2%}")

best_rf = grid_search_rf.best_estimator_
y_pred_best_rf = best_rf.predict(X_test_scaled)
print(f"Optimized Test Accuracy: {accuracy_score(y_test, y_pred_best_rf):.2%}")

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best RF Parameters: {'max_depth': 9, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 100}
Best CV Accuracy: 66.17%
Optimized Test Accuracy: 69.57%


In [52]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


xgb = XGBClassifier(random_state=2026, eval_metric='mlogloss')

param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9]
}

grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid_xgb, cv=5, n_jobs=-1, verbose=1)
grid_search_xgb.fit(X_train_scaled, y_train)

print(f"Best XGB Parameters: {grid_search_xgb.best_params_}")
print(f"Best XGB Cross-Validation Accuracy: {grid_search_xgb.best_score_:.2%}")

best_xgb = grid_search_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test_scaled)
print(f"XGBoost Optimized Test Accuracy: {accuracy_score(y_test, y_pred_xgb):.2%}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best XGB Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.9}
Best XGB Cross-Validation Accuracy: 66.03%
XGBoost Optimized Test Accuracy: 69.02%
